## Import Modules 

In [1]:
%matplotlib inline
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from sklearn.datasets import load_svmlight_files
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from pprint import pprint
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import make_moons
from sklearn.preprocessing import binarize
from time import time
from sklearn.model_selection import GridSearchCV
print('Import Done')


Import Done


## Read DataSets

In [2]:
print('Reading datasets...')
load_files = ["/data/aclImdb/train/labeledBow.feat","/data/aclImdb/test/labeledBow.feat"]
#importing files as string for TF_IDF
train_review,train_rating,test_review,test_rating = load_svmlight_files(load_files,n_features=None, dtype=str, multilabel=False, zero_based='auto', query_id=False)
print('Done')

Reading datasets...
Done


## Printing first and last Training records to check

In [3]:
#Print this if wnat to see the ratings labels before conversion i.e.0-9
print("First 10 Training ratings:")
for i in range(0,10):
    print(train_rating[i])
print("Last 10 Training ratings:")
for i in range(24990,25000):
    print(train_rating[i])
print("Last 10 Test ratings:")
for i in range(0,10):
    print(test_rating[i])
print("Last 10 Test ratings:")
for i in range(24990,25000):
    print(test_rating[i])

First 10 Training ratings:
9.0
7.0
9.0
10.0
8.0
10.0
10.0
7.0
7.0
7.0
Last 10 Training ratings:
1.0
2.0
4.0
1.0
1.0
1.0
1.0
4.0
2.0
2.0
Last 10 Test ratings:
10.0
10.0
7.0
7.0
10.0
7.0
7.0
9.0
9.0
7.0
Last 10 Test ratings:
4.0
4.0
4.0
4.0
2.0
3.0
3.0
4.0
1.0
1.0


## Converting to binary values

In [4]:
#manually iterating thorough all the records to convert all values to binary i.e 0 or 1
binary_train_target = []
binary_test_target = []
for i in range(len(train_rating)):
    if train_rating[i] > 5:
        binary_train_target.append(1)
    elif train_rating[i] <= 5:
        binary_train_target.append(0)
for i in range(len(test_rating)):
    if test_rating[i] > 5:
        binary_test_target.append(1)
    elif train_rating[i] <= 5:
        binary_test_target.append(0)
print("Converted to binary values")

Converted to binary values


## Printing first and last Training records After conversion

In [5]:
#print this to check if all values are converted to binary 
print("First 10 Training ratings:")
for i in range(0,10):
    print(binary_train_target[i])
print("Last 10 Training ratings:")
for i in range(24990,25000):
    print(binary_train_target[i])
print("First 10 Test ratings:")
for i in range(0,10):
    print(binary_test_target[i])
print("Last 10 Test ratings:")
for i in range(24990,25000):
    print(binary_test_target[i])

First 10 Training ratings:
1
1
1
1
1
1
1
1
1
1
Last 10 Training ratings:
0
0
0
0
0
0
0
0
0
0
First 10 Test ratings:
1
1
1
1
1
1
1
1
1
1
Last 10 Test ratings:
0
0
0
0
0
0
0
0
0
0


## Count Vectorizer

In [6]:
print('Vectorizing')
count_vect_test = CountVectorizer()
X_train_counts = count_vect_test.fit_transform(train_review.data)
print('Done!')
X_train_counts.shape


Vectorizing
Done!


(3456685, 94)

In [7]:
print('Vectorizing..')
count_vect_test = CountVectorizer()
X_test_counts = count_vect_test.fit_transform(test_review.data)
print('Done!')
X_test_counts.shape


Vectorizing..
Done!


(3340173, 86)

## TF-IDF

In [8]:
tf_transformer = TfidfTransformer().fit(train_review)
X_train_tf = tf_transformer.transform(train_review)
print('Done tf for train array')

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(train_review)
print('Done tfidf for train array')

tf_transformer = TfidfTransformer().fit(test_review)
X_test_tf = tf_transformer.transform(test_review)
print('Done tf for test array') 

tfidf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.fit_transform(test_review)
print('Done tfidf for train array')
# As the files loaded were in sting now we need float type
print('Now converting string files to float..')
test_review = test_review.astype(np.float)
train_review = train_review.astype(np.float)
print("Done Conversion")


Done tf for train array
Done tfidf for train array
Done tf for test array
Done tfidf for train array
Now converting string files to float..
Done Conversion


## Classifier Training 

### Linear SVM

In [9]:
clf_linear = LinearSVC()
print("Training On Linear SVM..")

start = time()
clf_linear.fit(X_train_tfidf,binary_train_target)
end = time()

print("Done Training")
clf_linear_accuracy = clf_linear.score(X_test_tfidf, binary_test_target)*100

print('Accuracy for Linear SVM Classifier: %.2f%% in %.2f Seconds'%(clf_linear_accuracy, end-start))

Training On Linear SVM..
Done Training
Accuracy for Linear SVM Classifier: 87.93% in 1.68 Seconds


### Naive bayes Multinominal

In [10]:
from sklearn.naive_bayes import MultinomialNB
clf_mnb = MultinomialNB()
print("Starting")
start = time()
clf_mnb = clf_mnb.fit(X_train_tfidf,binary_train_target)
end = time()

clf_mnb_accuracy = clf_mnb.score(X_test_tfidf, binary_test_target)*100
print('Accuracy for naive bayes: %.2f%% in %.2f Seconds'% (clf_mnb_accuracy, end-start))

Starting
Accuracy for naive bayes: 83.07% in 0.20 Seconds


### Naive bayes Bernoulli

In [11]:
from sklearn.naive_bayes import BernoulliNB
clf_bnb = BernoulliNB()
print("Starting")
start = time()
clf_bnb = clf_bnb.fit(X_train_tfidf,binary_train_target)
end = time()

clf_bnb_accuracy = clf_bnb.score(X_test_tfidf, binary_test_target)*100
print('Accuracy for naive bayes: %.2f%% in %.2f Seconds'% (clf_bnb_accuracy, end-start))


Starting
Accuracy for naive bayes: 82.74% in 0.32 Seconds


### Decision Tree classifier

In [12]:
from sklearn.tree import DecisionTreeClassifier
depth = [2,4,6,8,10,16,20,26,30,40]
for i in depth:

    clf_tree = DecisionTreeClassifier(max_depth=i)
    start = time()
    clf_tree.fit(X_train_tfidf, binary_train_target)
    end = time()

    clf_tree_accuracy = clf_tree.score(X_test_tfidf, binary_test_target)*100

    print('Accuracy = %.2f%% in %.2f Seconds with %.2f depth'  % (clf_tree_accuracy,end-start,i))
print('Done')


Accuracy = 65.40% in 2.53 Seconds with 2.00 depth
Accuracy = 68.59% in 4.63 Seconds with 4.00 depth
Accuracy = 70.44% in 7.45 Seconds with 6.00 depth
Accuracy = 71.87% in 11.17 Seconds with 8.00 depth
Accuracy = 72.52% in 15.77 Seconds with 10.00 depth
Accuracy = 72.51% in 33.23 Seconds with 16.00 depth
Accuracy = 72.05% in 44.82 Seconds with 20.00 depth
Accuracy = 71.68% in 53.83 Seconds with 26.00 depth
Accuracy = 71.66% in 58.27 Seconds with 30.00 depth
Accuracy = 70.99% in 73.19 Seconds with 40.00 depth


## Parameter Tunning 

In [13]:
#Setting alpha for bernoulli classifier using Grid Search to find best estimator
alphas = np.arange(8,9.5,0.01)
print('Tunning Parameter...')
clf_g =  GridSearchCV(estimator=BernoulliNB(),param_grid=dict(alpha=alphas))
clf_g.fit(X_train_tfidf,binary_train_target)
print('Done')
#save best parameter
g_best_alpha = clf_g.best_estimator_



Tunning Parameter...
Done


In [14]:
g_best_alpha.fit(X_train_tfidf, binary_train_target)
accu = g_best_alpha.score(X_test_tfidf, binary_test_target)*100

print('Test accuracy Before tunning',clf_bnb_accuracy, 'and afetr parameter tunning= ',accu)

Test accuracy Before tunning 82.736 and afetr parameter tunning=  82.924


## Final prediction using bernoulli with best alpha on test set and saving file 

In [15]:
final_array = []
file_name = 'final_review.csv'# Change file name here 
print('Predicting by burnolis...')
predicted = g_best_alpha.predict(test_review)
print('Done Prediction!')
#Adding positive or negative with index of predicted file
for i in range(len(predicted)):
    if predicted[i] == 0:
        final_array.append([i,'Positive'])

    elif predicted[i] == 1:
        final_array.append([i,'Negative'])
#Using pandas library to save file as csv
df = pd.DataFrame(data=final_array)
df.columns = ['Movie','Rating']
df.to_csv(file_name,index=False)
print('file saved as:' + file_name)

Predicting by burnolis...
Done Prediction!
file saved as:final_review.csv


In [16]:
#Check if file saved is in right format
test = pd.read_csv(file_name)
test

,Movie,Rating
0,0,Negative
1,1,Negative
2,2,Positive
3,3,Positive
4,4,Negative
5,5,Positive
6,6,Negative
7,7,Negative
8,8,Positive
9,9,Negative


## Appendix
#### Other testing methods used for project
##### Parameter Tuning for linear SVM

In [ ]:
clist = 2**np.array(range(-2, 10), dtype='float')
cvscores = []
for c in clist:
    print('vlaue for C is:')
    print(c)
    clf = LinearSVC(C=c)
    scores = cross_val_score(clf, X_train_tfidf, binary_train_target, cv=3)
    print(scores)
    cvscores.append(scores.mean()*100)
    bestscore, bestC = max([(val, clist[idx]) for (idx, val) in enumerate(cvscores)])

print('Best CV accuracy =', bestscore, 'achieved at C =', bestC)

clf = LinearSVC(C=bestC)
clf.fit(X_train_tfidf, binary_train_target)
accu = clf.score(X_test_tfidf, binary_test_target)*100
print('Test accuracy =', accu, 'achieved at C =', bestC)

In [ ]:
#Setting alpha for bernoulli classifier using Grid Search to find best estimator
alphas = 2**np.array(range(-2, 10), dtype='float')
print('Tunning Parameter...')
clf_g =  GridSearchCV(estimator=BernoulliNB(),param_grid=dict(alpha=alphas))
clf_g.fit(X_train_tfidf,binary_train_target)
print('Done')
#save best parameter
g_best_alpha = clf_g.best_estimator_
print(g_best_alpha)



### Printing best estimator gives alpha value as 8 so further using cross validation with range 8 to 9 with increment 0.01

In [ ]:
alphas = np.arange(8,9.5,0.01)
mean_scores = []
print("tunning....")

for i in alphas:
    clf_g =  BernoulliNB(alpha=i)
    scores = cross_val_score(clf_g, train_review, binary_train_target, cv=3)
    mean_scores.append(scores.mean()*100)
    bestmean, bestAlpha = max([(val, alphas[idx]) for (idx, val) in enumerate(mean_scores)])
print('Done')
clf = BernoulliNB(alpha=i)
clf.fit(X_train_tfidf, binary_train_target)
accu = clf.score(X_test_tfidf, binary_test_target)*100
print('Test accuracy =', accu, 'achieved at alpha =', bestAlpha)

##### References 